https://onlinezhuanjia.com/python-facebook-page-scraper-tool/
https://www.quintly.com/
    
    
    
    

In [2]:
import json
import datetime
import csv
import time
try:
    from urllib.request import urlopen, Request
except ImportError:
    from urllib2 import urlopen, Request
 
#app_id = "&amp;amp;amp;lt;FILL IN&amp;amp;amp;gt;"
#app_secret = "&amp;amp;amp;lt;FILL IN&amp;amp;amp;gt;"  # DO NOT SHARE WITH ANYONE!
page_id = "barackobama"
 
# input date formatted as YYYY-MM-DD
since_date = "2018-01-01"
until_date = "2018-04-30"
 
#access_token = app_id + "|" + app_secret
access_token = "EAADfPdAMIXQBALanmEmEEgETIr01T7N7pZCMsxcQvLIkj3p6qhjaMEJXYFVfoNRIpIlUGZA0HZCQIuRWgkKO3KUykXLFWEq5d0BdTkeGwQiqX360vpp1Ah9Jrs3w0YYo7FMIJ8T7WVQtX2oyqYeDvGtdaRJNZCDXqaBCIuzHCeBSxJP4PI3JVVwaLCiSiagZD"
 
def request_until_succeed(url):
    req = Request(url)
    success = False
    while success is False:
        try:
            response = urlopen(req)
            if response.getcode() == 200:
                success = True
        except Exception as e:
            print(e)
            time.sleep(5)
 
            print("Error for URL {}: {}".format(url, datetime.datetime.now()))
            print("Retrying.")
 
    return response.read()
 
 
# Needed to write tricky unicode correctly to csv
def unicode_decode(text):
    try:
        return text.encode('utf-8').decode()
    except UnicodeDecodeError:
        return text.encode('utf-8')
 
 
def getFacebookPageFeedUrl(base_url):
 
    # Construct the URL string; see http://stackoverflow.com/a/37239851 for
    # Reactions parameters
    fields = "&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id," + \
        "comments.limit(0).summary(true),shares,reactions" + \
        ".limit(0).summary(true)"
 
    return base_url + fields
 
 
def getReactionsForStatuses(base_url):
 
    reaction_types = ['like', 'love', 'wow', 'haha', 'sad', 'angry']
    reactions_dict = {}   # dict of {status_id: tuple&amp;amp;amp;lt;6&amp;amp;amp;gt;}
 
    for reaction_type in reaction_types:
        fields = "&amp;amp;amp;amp;fields=reactions.type({}).limit(0).summary(total_count)".format(
            reaction_type.upper())
 
        url = base_url + fields
 
        data = json.loads(request_until_succeed(url))['data']
 
        data_processed = set()  # set() removes rare duplicates in statuses
        for status in data:
            id = status['id']
            count = status['reactions']['summary']['total_count']
            data_processed.add((id, count))
 
        for id, count in data_processed:
            if id in reactions_dict:
                reactions_dict[id] = reactions_dict[id] + (count,)
            else:
                reactions_dict[id] = (count,)
 
    return reactions_dict
 
 
def processFacebookPageFeedStatus(status):
 
    # The status is now a Python dictionary, so for top-level items,
    # we can simply call the key.
 
    # Additionally, some items may not always exist,
    # so must check for existence first
 
    status_id = status['id']
    status_type = status['type']
 
    status_message = '' if 'message' not in status else \
        unicode_decode(status['message'])
    link_name = '' if 'name' not in status else \
        unicode_decode(status['name'])
    status_link = '' if 'link' not in status else \
        unicode_decode(status['link'])
    status_permalink_url = '' if 'permalink_url' not in status.keys() else \
            unicode_decode(status['permalink_url'])
 
    # Time needs special care since a) it's in UTC and
    # b) it's not easy to use in statistical programs.
 
    status_published = datetime.datetime.strptime(
        status['created_time'], '%Y-%m-%dT%H:%M:%S+0000')
    status_published = status_published + \
        datetime.timedelta(hours=-5)  # EST
    status_published = status_published.strftime(
        '%Y-%m-%d %H:%M:%S')  # best time format for spreadsheet programs
 
    # Nested items require chaining dictionary keys.
 
    num_reactions = 0 if 'reactions' not in status else \
        status['reactions']['summary']['total_count']
    num_comments = 0 if 'comments' not in status else \
        status['comments']['summary']['total_count']
    num_shares = 0 if 'shares' not in status else status['shares']['count']
 
    return (status_id, status_message, link_name, status_type, status_link,status_permalink_url,
            status_published, num_reactions, num_comments, num_shares)
 
 
def scrapeFacebookPageFeedStatus(page_id, access_token, since_date, until_date):
    with open('{}_facebook_statuses.csv'.format(page_id), 'w',encoding="utf-8") as file:
        w = csv.writer(file)
        w.writerow(["status_id", "status_message", "link_name", "status_type",
                    "status_link","permalink_url", "status_published", "num_reactions",
                    "num_comments", "num_shares", "num_likes", "num_loves",
                    "num_wows", "num_hahas", "num_sads", "num_angrys",
                    "num_special"])
 
        has_next_page = True
        num_processed = 0
        scrape_starttime = datetime.datetime.now()
        after = ''
        base = "https://graph.facebook.com/v2.9"
        node = "/{}/posts".format(page_id)
        parameters = "/?limit={}&amp;amp;amp;amp;access_token={}".format(100, access_token)
        since = "&amp;amp;amp;amp;since={}".format(since_date) if since_date \
            is not '' else ''
        until = "&amp;amp;amp;amp;until={}".format(until_date) if until_date \
            is not '' else ''
 
        print("Scraping {} Facebook Page: {}\n".format(page_id, scrape_starttime))
 
        while has_next_page:
            after = '' if after is '' else "&amp;amp;amp;amp;after={}".format(after)
            base_url = base + node + parameters + after + since + until
 
            url = getFacebookPageFeedUrl(base_url)
            statuses = json.loads(request_until_succeed(url))
            reactions = getReactionsForStatuses(base_url)
 
            for status in statuses['data']:
 
                # Ensure it is a status with the expected metadata
                if 'reactions' in status:
                    status_data = processFacebookPageFeedStatus(status)
                    reactions_data = reactions[status_data[0]]
 
                    # calculate thankful/pride through algebra
                    num_special = status_data[7] - sum(reactions_data)
                    w.writerow(status_data + reactions_data + (num_special,))
 
                num_processed += 1
                if num_processed % 100 == 0:
                    print("{} Statuses Processed: {}".format
                          (num_processed, datetime.datetime.now()))
 
            # if there is no next page, we're done.
            if 'paging' in statuses:
                after = statuses['paging']['cursors']['after']
            else:
                has_next_page = False
 
        print("\nDone!\n{} Statuses Processed in {}".format(
              num_processed, datetime.datetime.now() - scrape_starttime))
 
 
if __name__ == '__main__':
    scrapeFacebookPageFeedStatus(page_id, access_token, since_date, until_date)

Scraping barackobama Facebook Page: 2020-03-20 11:44:31.690655

HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/barackobama/posts/?limit=100&amp;amp;amp;amp;access_token=EAADfPdAMIXQBALanmEmEEgETIr01T7N7pZCMsxcQvLIkj3p6qhjaMEJXYFVfoNRIpIlUGZA0HZCQIuRWgkKO3KUykXLFWEq5d0BdTkeGwQiqX360vpp1Ah9Jrs3w0YYo7FMIJ8T7WVQtX2oyqYeDvGtdaRJNZCDXqaBCIuzHCeBSxJP4PI3JVVwaLCiSiagZD&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-04-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 11:44:36.940207
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/barackobama/posts/?limit=100&amp;amp;amp;amp;access_token=EAADfPdAMIXQBALanmEmEEgETIr01T7N7pZCMsxcQvLIkj3p6qhjaMEJXYFVfoNRIpIlUGZA0HZCQIuRWgkKO3KUykXLFWEq5d0BdTkeGwQiqX360vpp1Ah9Jrs3w0YYo7FMIJ8T7WVQtX2oyqYeDvGtdaRJNZCDXqaBCIuzHCeBSxJP4PI3JVVwaLCiSiagZD&amp;amp;amp;amp;since=2018-01-

Error for URL https://graph.facebook.com/v2.9/barackobama/posts/?limit=100&amp;amp;amp;amp;access_token=EAADfPdAMIXQBALanmEmEEgETIr01T7N7pZCMsxcQvLIkj3p6qhjaMEJXYFVfoNRIpIlUGZA0HZCQIuRWgkKO3KUykXLFWEq5d0BdTkeGwQiqX360vpp1Ah9Jrs3w0YYo7FMIJ8T7WVQtX2oyqYeDvGtdaRJNZCDXqaBCIuzHCeBSxJP4PI3JVVwaLCiSiagZD&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-04-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 11:45:49.960198
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/barackobama/posts/?limit=100&amp;amp;amp;amp;access_token=EAADfPdAMIXQBALanmEmEEgETIr01T7N7pZCMsxcQvLIkj3p6qhjaMEJXYFVfoNRIpIlUGZA0HZCQIuRWgkKO3KUykXLFWEq5d0BdTkeGwQiqX360vpp1Ah9Jrs3w0YYo7FMIJ8T7WVQtX2oyqYeDvGtdaRJNZCDXqaBCIuzHCeBSxJP4PI3JVVwaLCiSiagZD&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-04-30&amp;amp;amp;amp;fields=message,link,permalink_url,create

Error for URL https://graph.facebook.com/v2.9/barackobama/posts/?limit=100&amp;amp;amp;amp;access_token=EAADfPdAMIXQBALanmEmEEgETIr01T7N7pZCMsxcQvLIkj3p6qhjaMEJXYFVfoNRIpIlUGZA0HZCQIuRWgkKO3KUykXLFWEq5d0BdTkeGwQiqX360vpp1Ah9Jrs3w0YYo7FMIJ8T7WVQtX2oyqYeDvGtdaRJNZCDXqaBCIuzHCeBSxJP4PI3JVVwaLCiSiagZD&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-04-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 11:47:08.117826
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/barackobama/posts/?limit=100&amp;amp;amp;amp;access_token=EAADfPdAMIXQBALanmEmEEgETIr01T7N7pZCMsxcQvLIkj3p6qhjaMEJXYFVfoNRIpIlUGZA0HZCQIuRWgkKO3KUykXLFWEq5d0BdTkeGwQiqX360vpp1Ah9Jrs3w0YYo7FMIJ8T7WVQtX2oyqYeDvGtdaRJNZCDXqaBCIuzHCeBSxJP4PI3JVVwaLCiSiagZD&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-04-30&amp;amp;amp;amp;fields=message,link,permalink_url,create

Error for URL https://graph.facebook.com/v2.9/barackobama/posts/?limit=100&amp;amp;amp;amp;access_token=EAADfPdAMIXQBALanmEmEEgETIr01T7N7pZCMsxcQvLIkj3p6qhjaMEJXYFVfoNRIpIlUGZA0HZCQIuRWgkKO3KUykXLFWEq5d0BdTkeGwQiqX360vpp1Ah9Jrs3w0YYo7FMIJ8T7WVQtX2oyqYeDvGtdaRJNZCDXqaBCIuzHCeBSxJP4PI3JVVwaLCiSiagZD&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-04-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 11:48:26.337586
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/barackobama/posts/?limit=100&amp;amp;amp;amp;access_token=EAADfPdAMIXQBALanmEmEEgETIr01T7N7pZCMsxcQvLIkj3p6qhjaMEJXYFVfoNRIpIlUGZA0HZCQIuRWgkKO3KUykXLFWEq5d0BdTkeGwQiqX360vpp1Ah9Jrs3w0YYo7FMIJ8T7WVQtX2oyqYeDvGtdaRJNZCDXqaBCIuzHCeBSxJP4PI3JVVwaLCiSiagZD&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-04-30&amp;amp;amp;amp;fields=message,link,permalink_url,create

HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/barackobama/posts/?limit=100&amp;amp;amp;amp;access_token=EAADfPdAMIXQBALanmEmEEgETIr01T7N7pZCMsxcQvLIkj3p6qhjaMEJXYFVfoNRIpIlUGZA0HZCQIuRWgkKO3KUykXLFWEq5d0BdTkeGwQiqX360vpp1Ah9Jrs3w0YYo7FMIJ8T7WVQtX2oyqYeDvGtdaRJNZCDXqaBCIuzHCeBSxJP4PI3JVVwaLCiSiagZD&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-04-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 11:49:44.542501
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/barackobama/posts/?limit=100&amp;amp;amp;amp;access_token=EAADfPdAMIXQBALanmEmEEgETIr01T7N7pZCMsxcQvLIkj3p6qhjaMEJXYFVfoNRIpIlUGZA0HZCQIuRWgkKO3KUykXLFWEq5d0BdTkeGwQiqX360vpp1Ah9Jrs3w0YYo7FMIJ8T7WVQtX2oyqYeDvGtdaRJNZCDXqaBCIuzHCeBSxJP4PI3JVVwaLCiSiagZD&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-04-30&amp;amp;amp;amp;fields=messa

...

HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/barackobama/posts/?limit=100&amp;amp;amp;amp;access_token=EAADfPdAMIXQBALanmEmEEgETIr01T7N7pZCMsxcQvLIkj3p6qhjaMEJXYFVfoNRIpIlUGZA0HZCQIuRWgkKO3KUykXLFWEq5d0BdTkeGwQiqX360vpp1Ah9Jrs3w0YYo7FMIJ8T7WVQtX2oyqYeDvGtdaRJNZCDXqaBCIuzHCeBSxJP4PI3JVVwaLCiSiagZD&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-04-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 12:08:04.598865
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/barackobama/posts/?limit=100&amp;amp;amp;amp;access_token=EAADfPdAMIXQBALanmEmEEgETIr01T7N7pZCMsxcQvLIkj3p6qhjaMEJXYFVfoNRIpIlUGZA0HZCQIuRWgkKO3KUykXLFWEq5d0BdTkeGwQiqX360vpp1Ah9Jrs3w0YYo7FMIJ8T7WVQtX2oyqYeDvGtdaRJNZCDXqaBCIuzHCeBSxJP4PI3JVVwaLCiSiagZD&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-04-30&amp;amp;amp;amp;fields=messa

HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/barackobama/posts/?limit=100&amp;amp;amp;amp;access_token=EAADfPdAMIXQBALanmEmEEgETIr01T7N7pZCMsxcQvLIkj3p6qhjaMEJXYFVfoNRIpIlUGZA0HZCQIuRWgkKO3KUykXLFWEq5d0BdTkeGwQiqX360vpp1Ah9Jrs3w0YYo7FMIJ8T7WVQtX2oyqYeDvGtdaRJNZCDXqaBCIuzHCeBSxJP4PI3JVVwaLCiSiagZD&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-04-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 12:09:23.039683
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/barackobama/posts/?limit=100&amp;amp;amp;amp;access_token=EAADfPdAMIXQBALanmEmEEgETIr01T7N7pZCMsxcQvLIkj3p6qhjaMEJXYFVfoNRIpIlUGZA0HZCQIuRWgkKO3KUykXLFWEq5d0BdTkeGwQiqX360vpp1Ah9Jrs3w0YYo7FMIJ8T7WVQtX2oyqYeDvGtdaRJNZCDXqaBCIuzHCeBSxJP4PI3JVVwaLCiSiagZD&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-04-30&amp;amp;amp;amp;fields=messa

Error for URL https://graph.facebook.com/v2.9/barackobama/posts/?limit=100&amp;amp;amp;amp;access_token=EAADfPdAMIXQBALanmEmEEgETIr01T7N7pZCMsxcQvLIkj3p6qhjaMEJXYFVfoNRIpIlUGZA0HZCQIuRWgkKO3KUykXLFWEq5d0BdTkeGwQiqX360vpp1Ah9Jrs3w0YYo7FMIJ8T7WVQtX2oyqYeDvGtdaRJNZCDXqaBCIuzHCeBSxJP4PI3JVVwaLCiSiagZD&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-04-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 12:10:42.288979
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/barackobama/posts/?limit=100&amp;amp;amp;amp;access_token=EAADfPdAMIXQBALanmEmEEgETIr01T7N7pZCMsxcQvLIkj3p6qhjaMEJXYFVfoNRIpIlUGZA0HZCQIuRWgkKO3KUykXLFWEq5d0BdTkeGwQiqX360vpp1Ah9Jrs3w0YYo7FMIJ8T7WVQtX2oyqYeDvGtdaRJNZCDXqaBCIuzHCeBSxJP4PI3JVVwaLCiSiagZD&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-04-30&amp;amp;amp;amp;fields=message,link,permalink_url,create

Error for URL https://graph.facebook.com/v2.9/barackobama/posts/?limit=100&amp;amp;amp;amp;access_token=EAADfPdAMIXQBALanmEmEEgETIr01T7N7pZCMsxcQvLIkj3p6qhjaMEJXYFVfoNRIpIlUGZA0HZCQIuRWgkKO3KUykXLFWEq5d0BdTkeGwQiqX360vpp1Ah9Jrs3w0YYo7FMIJ8T7WVQtX2oyqYeDvGtdaRJNZCDXqaBCIuzHCeBSxJP4PI3JVVwaLCiSiagZD&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-04-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 12:12:00.449484
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/barackobama/posts/?limit=100&amp;amp;amp;amp;access_token=EAADfPdAMIXQBALanmEmEEgETIr01T7N7pZCMsxcQvLIkj3p6qhjaMEJXYFVfoNRIpIlUGZA0HZCQIuRWgkKO3KUykXLFWEq5d0BdTkeGwQiqX360vpp1Ah9Jrs3w0YYo7FMIJ8T7WVQtX2oyqYeDvGtdaRJNZCDXqaBCIuzHCeBSxJP4PI3JVVwaLCiSiagZD&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-04-30&amp;amp;amp;amp;fields=message,link,permalink_url,create

KeyboardInterrupt: 